# SQL Data Loading - Star Schema for Business Intelligence

This notebook loads cleaned data into a SQL database using a **Star Schema** dimensional model optimized for BI and analytics.

## Architecture:
```
Cleaned CSVs → Star Schema (SQLite)
                  ├── Fact: fact_sales
                  └── Dimensions:
                      ├── dim_customer
                      ├── dim_product
                      ├── dim_date
                      ├── dim_geography
                      └── dim_order
```

# 01. Setup & Database Connection

In [8]:
# 01. Setup & Libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, text
from datetime import datetime
import os
from dotenv import load_dotenv
from pathlib import Path

# Load environment variables from project root
# Find the .env file in the parent directory (project root)
env_path = Path('..') / '.env'

# IMPORTANT: override=True forces reload even if variables already exist
load_dotenv(dotenv_path=env_path, override=True)

print(f"Loading .env from: {env_path.absolute()}")
print(f"File exists: {env_path.exists()}")

# Supabase Database Configuration
DATABASE_URL = os.getenv('DATABASE_URL')

# Alternative: Build connection string from individual components
if not DATABASE_URL:
    SUPABASE_HOST = os.getenv('SUPABASE_HOST')
    SUPABASE_PORT = os.getenv('SUPABASE_PORT', '5432')
    SUPABASE_DATABASE = os.getenv('SUPABASE_DATABASE', 'postgres')
    SUPABASE_USER = os.getenv('SUPABASE_USER')
    SUPABASE_PASSWORD = os.getenv('SUPABASE_PASSWORD')
    
    # Debug: Check if variables are loaded
    print(f"\nLoaded credentials:")
    print(f"Host: {SUPABASE_HOST}")
    print(f"User: {SUPABASE_USER}")
    print(f"Port: {SUPABASE_PORT}")
    print(f"Password: {'*' * len(SUPABASE_PASSWORD) if SUPABASE_PASSWORD else 'NOT SET'}")
    
    if not all([SUPABASE_HOST, SUPABASE_USER, SUPABASE_PASSWORD]):
        raise ValueError("❌ Missing Supabase credentials in .env file!")
    
    DATABASE_URL = f"postgresql://{SUPABASE_USER}:{SUPABASE_PASSWORD}@{SUPABASE_HOST}:{SUPABASE_PORT}/{SUPABASE_DATABASE}"

# Create SQLAlchemy engine
engine = create_engine(DATABASE_URL)

# Test connection
print("\nTesting connection...")
try:
    with engine.connect() as conn:
        result = conn.execute(text("SELECT version();"))
        version = result.fetchone()[0]
        print("✅ Connected to Supabase PostgreSQL!")
        print(f"Database version: {version[:80]}...")
except Exception as e:
    print(f"❌ Connection failed: {e}")
    print("\n💡 Troubleshooting:")
    print("   1. Verify your password is correct in .env")
    print("   2. Check Supabase dashboard for connection string")
    print("   3. Ensure you're using Session Pooler (port 6543)")
    print("   4. Try restarting the Jupyter kernel")


Loading .env from: /Users/diegoferra/Documents/Python codes/bloque_clase/notebooks/../.env
File exists: True

Loaded credentials:
Host: aws-1-us-east-1.pooler.supabase.com
User: postgres.ypznufmiuekmrtdjmcux
Port: 6543
Password: *********

Testing connection...
✅ Connected to Supabase PostgreSQL!
Database version: PostgreSQL 17.6 on aarch64-unknown-linux-gnu, compiled by gcc (GCC) 13.2.0, 64-b...


In [9]:

# Load cleaned datasets (uncomment when ready)
customerAddress = pd.read_csv('../data/processed/clean_CustomerAddress.csv')
individualCustomer = pd.read_csv('../data/processed/clean_IndividualCustomer.csv')
productCatalog = pd.read_csv('../data/processed/clean_ProductCatalog.csv')
ordersList = pd.read_csv('../data/processed/clean_OrdersList.csv')
generalOrder = pd.read_csv('../data/processed/clean_GeneralOrderDetail.csv')
productOrderDetail = pd.read_csv('../data/processed/clean_ProductOrderDetail.csv')

/var/folders/jn/dc5313mj0qs72qr08qqw3w3r0000gn/T/ipykernel_36285/2268709587.py:2: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  customerAddress = pd.read_csv('../data/processed/clean_CustomerAddress.csv')
/var/folders/jn/dc5313mj0qs72qr08qqw3w3r0000gn/T/ipykernel_36285/2268709587.py:3: DtypeWarning: Columns (7,8,23) have mixed types. Specify dtype option on import or set low_memory=False.
  individualCustomer = pd.read_csv('../data/processed/clean_IndividualCustomer.csv')
/var/folders/jn/dc5313mj0qs72qr08qqw3w3r0000gn/T/ipykernel_36285/2268709587.py:5: DtypeWarning: Columns (8,11,21) have mixed types. Specify dtype option on import or set low_memory=False.
  ordersList = pd.read_csv('../data/processed/clean_OrdersList.csv')
/var/folders/jn/dc5313mj0qs72qr08qqw3w3r0000gn/T/ipykernel_36285/2268709587.py:6: DtypeWarning: Columns (8,32) have mixed types. Specify dtype option on import or set low_memory=False.
  generalOrder = pd.read_

# 02. Star Schema Design

## Dimensional Model Overview

The star schema consists of one **fact table** surrounded by **dimension tables**. This design optimizes query performance for analytical workloads and BI dashboards.

## 📊 Fact Table: `fact_sales`

**Grain:** One row per product item in an order

**Purpose:** Stores transactional sales data with foreign keys to dimensions

### Schema:

| Column | Type | Description |
|--------|------|-------------|
| `sale_id` | INTEGER PRIMARY KEY | Surrogate key (auto-increment) |
| `order_id` | VARCHAR(50) | Business key - Order identifier |
| `customer_key` | INTEGER FK | → dim_customer |
| `product_key` | INTEGER FK | → dim_product |
| `date_key` | INTEGER FK | → dim_date (YYYYMMDD format) |
| `geography_key` | INTEGER FK | → dim_geography |
| `order_key` | INTEGER FK | → dim_order |
| **Measures (Metrics):** | | |
| `quantity` | INTEGER | Units sold |
| `unit_price` | DECIMAL(10,2) | Price per unit |
| `list_price` | DECIMAL(10,2) | Original list price |
| `selling_price` | DECIMAL(10,2) | Final selling price |
| `discount_amount` | DECIMAL(10,2) | Discount applied |
| `shipping_price` | DECIMAL(10,2) | Shipping cost |
| `total_amount` | DECIMAL(10,2) | Total transaction value |
| `is_gift` | BOOLEAN | Gift flag |

**Source Tables:** `productOrderDetail` (primary), `ordersList`, `generalOrder`

## 👤 Dimension: `dim_customer`

**Purpose:** Customer profile and demographic information

### Schema:

| Column | Type | Description |
|--------|------|-------------|
| `customer_key` | INTEGER PRIMARY KEY | Surrogate key |
| `user_id` | VARCHAR(50) UNIQUE | Business key |
| `birth_date` | DATE | Date of birth |
| `customer_age` | INTEGER | Calculated age |
| `gender` | VARCHAR(10) | Customer gender |
| `email` | VARCHAR(255) | Email address |
| `phone` | VARCHAR(50) | Phone number |
| `first_purchase_date` | DATE | Date of first purchase |
| `last_session_date` | DATETIME | Last platform activity |
| `is_active` | BOOLEAN | Active customer flag |
| `created_at` | DATETIME | Record creation timestamp |

**Source Table:** `individualCustomer`

**SCD Type:** Type 1 (overwrite) - For this project, we assume customer data doesn't need historical tracking

## 🛒 Dimension: `dim_product`

**Purpose:** Product catalog and hierarchy information

### Schema:

| Column | Type | Description |
|--------|------|-------------|
| `product_key` | INTEGER PRIMARY KEY | Surrogate key |
| `product_id` | VARCHAR(50) UNIQUE | Business key (IdMaterial) |
| `product_name` | VARCHAR(255) | Product material name |
| `ean_upc` | VARCHAR(50) | Barcode |
| `brand` | VARCHAR(100) | Product brand |
| `category` | VARCHAR(100) | Product category |
| `segment` | VARCHAR(100) | Product segment |
| `is_active` | BOOLEAN | Active in catalog |

**Source Table:** `productCatalog`

## 📅 Dimension: `dim_date`

**Purpose:** Time intelligence for temporal analysis

### Schema:

| Column | Type | Description |
|--------|------|-------------|
| `date_key` | INTEGER PRIMARY KEY | YYYYMMDD format (e.g., 20210115) |
| `full_date` | DATE UNIQUE | Actual date |
| `year` | INTEGER | Year (2021, 2022) |
| `quarter` | INTEGER | Quarter (1-4) |
| `month` | INTEGER | Month (1-12) |
| `month_name` | VARCHAR(20) | Month name (January, etc.) |
| `week_of_year` | INTEGER | ISO week number |
| `day_of_month` | INTEGER | Day (1-31) |
| `day_of_week` | INTEGER | Weekday (1=Monday, 7=Sunday) |
| `day_name` | VARCHAR(20) | Day name (Monday, etc.) |
| `is_weekend` | BOOLEAN | Weekend flag |
| `is_holiday` | BOOLEAN | Holiday flag (optional) |
| `quarter_name` | VARCHAR(10) | Q1, Q2, Q3, Q4 |
| `year_month` | VARCHAR(10) | YYYY-MM format |

**Source:** Generated programmatically from date range in data (Jan 2021 - Nov 2022)

**Note:** This is a conformed dimension - same date dimension used across all facts

## 📍 Dimension: `dim_geography`

**Purpose:** Location and address information for geographic analysis

### Schema:

| Column | Type | Description |
|--------|------|-------------|
| `geography_key` | INTEGER PRIMARY KEY | Surrogate key |
| `address_id` | VARCHAR(50) | Business key |
| `user_id` | VARCHAR(50) | Associated customer |
| `country` | VARCHAR(100) | Country name |
| `state` | VARCHAR(100) | State/province |
| `city` | VARCHAR(100) | City name |
| `neighborhood` | VARCHAR(100) | Neighborhood |
| `postal_code` | VARCHAR(20) | ZIP/postal code |
| `street` | VARCHAR(255) | Street address |
| `latitude` | DECIMAL(10,8) | Geographic coordinate |
| `longitude` | DECIMAL(11,8) | Geographic coordinate |
| `address_type` | VARCHAR(50) | Residential, commercial, etc. |
| `is_default` | BOOLEAN | Default address flag |

**Source Table:** `customerAddress`

## 📦 Dimension: `dim_order`

**Purpose:** Order-level attributes and status information

### Schema:

| Column | Type | Description |
|--------|------|-------------|
| `order_key` | INTEGER PRIMARY KEY | Surrogate key |
| `order_id` | VARCHAR(50) UNIQUE | Business key |
| `creation_date` | DATETIME | Order creation timestamp |
| `authorized_date` | DATETIME | Payment authorization |
| `invoiced_date` | DATETIME | Invoice date |
| `order_status` | VARCHAR(50) | Current status |
| `payment_method` | VARCHAR(50) | Payment type |
| `shipping_estimated_date` | DATE | Estimated delivery |
| `shipping_estimated_min` | DATE | Min delivery estimate |
| `shipping_estimated_max` | DATE | Max delivery estimate |
| `days_to_shipping` | INTEGER | Days from order to ship |
| `order_year` | INTEGER | Order year |
| `order_month` | INTEGER | Order month |
| `order_quarter` | INTEGER | Order quarter |
| `order_day_of_week` | INTEGER | Order weekday |
| `channel` | VARCHAR(50) | Sales channel |
| `seller_id` | VARCHAR(50) | Seller identifier |

**Source Tables:** `ordersList`, `generalOrder`

## 🔗 Relationships & Cardinality

```
dim_customer (1) ──────── (*) fact_sales
dim_product (1)  ──────── (*) fact_sales
dim_date (1)     ──────── (*) fact_sales
dim_geography (1)──────── (*) fact_sales
dim_order (1)    ──────── (*) fact_sales
```

**Key Points:**
- All relationships are **1:Many** (dimension → fact)
- Fact table contains **only foreign keys + measures**
- Dimensions are **denormalized** for query performance
- Date dimension is **pre-populated** with all dates in range

## 📝 Design Decisions & Notes

### 1. Grain Selection
- **Fact grain:** Product line item per order (most atomic level)
- Allows aggregation to any level: order, customer, product, day, etc.

### 2. Surrogate Keys
- All dimensions use auto-increment surrogate keys
- Business keys (userId, orderId, productId) preserved for reference
- Simplifies joins and improves performance

### 3. Slowly Changing Dimensions (SCD)
- **Type 1 (Overwrite)** for all dimensions
- No historical tracking needed for this project
- Future enhancement: Type 2 for customer/product changes

### 4. Degenerate Dimensions
- `order_id` stored in fact table (not just FK)
- Allows grouping by order without joining dim_order

### 5. Conformed Dimensions
- `dim_date` is a conformed dimension
- Can be reused across multiple fact tables if schema expands

### 6. Missing Data Handling
- Unknown/missing dimension values → special record with key = -1
- Example: Unknown customer, Unknown product, etc.

### 7. Data Types
- Decimals for monetary values (avoid floating point errors)
- VARCHAR with appropriate lengths
- DATE/DATETIME for temporal columns
- BOOLEAN for flags

## 🎯 Business Metrics Enabled by This Model

This star schema design enables analysis of:

**Sales Performance:**
- Total revenue by period/product/customer
- Average order value
- Discount effectiveness
- Shipping cost analysis

**Customer Analytics:**
- Customer lifetime value (CLV)
- Customer segmentation by age/geography
- Repeat purchase rate
- Customer acquisition trends

**Product Analytics:**
- Top products by revenue/quantity
- Category performance
- Brand comparison
- Product mix analysis

**Geographic Analytics:**
- Sales by country/state/city
- Regional performance
- Market penetration

**Temporal Analytics:**
- Seasonality patterns
- Year-over-year growth
- Weekend vs weekday sales
- Monthly/quarterly trends

**Operational Metrics:**
- Fulfillment time (days to shipping)
- Order status distribution
- Payment method preferences

# 03. Create SQL Tables (DDL)

In [10]:
# TODO: Section 3 - Create dimension and fact tables with DDL statements

# 04. Load Data into Star Schema

In [11]:
# TODO: Section 4 - Populate dimensions and fact table

# 05. Validation & Testing

In [12]:
# TODO: Section 5 - Validate data integrity and run test queries